In [1]:
import pandas as pd
import sweetviz as sv

In [10]:
from datetime import datetime

In [17]:
import time

In [24]:
time.strftime('%d/%m/%Y')

'05/01/2025'

In [16]:
datetime.date(datetime.now())

datetime.date(2025, 1, 5)

In [34]:
mono_a  = pd.read_csv("../data/MonobankTransactionHistory_ARTEM.csv").query("type == 'outcome' ")
mono_i  = pd.read_csv("../data/MonobankTransactionHistory_IRA.csv").query("type == 'outcome' ")
mono_tg = pd.read_csv("../data/MonoTransactionFromTelegram_ARTEM.csv")

In [35]:
transaction_history = pd.concat([mono_a[mono_tg.columns], mono_i[mono_tg.columns], mono_tg]) 

### EDA

In [36]:
transaction_history

,Unnamed: 0,time,description,amount,full_date,month_number,week_number
0,0,1732554999,Glovo,588,25.11.2024,11,48
1,1,1732489979,535129****2010,1837,25.11.2024,11,48
2,2,1732448089,Реберня в Тернополі,1689,24.11.2024,11,47
3,3,1732389399,414950****4044,2000,23.11.2024,11,47
4,4,1732382912,Glovo,607,23.11.2024,11,47
...,...,...,...,...,...,...,...
360,360,1732448655,Реберня в Тернополі,1689,24.11.2024,11,47
361,361,1732490130,535129****2010,1837,25.11.2024,11,48
362,362,1732555962,Glovo,588,25.11.2024,11,48
363,363,1732612762,ZERNO,120,26.11.2024,11,48


In [37]:
transaction_history['full_date'] = transaction_history['full_date'].apply(lambda x: x.replace('.','-'))

In [38]:
transaction_history['full_date']  = pd.to_datetime(transaction_history['full_date'])

C:\Users\vinew\AppData\Local\Temp\ipykernel_23772\4109452602.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  transaction_history['full_date']  = pd.to_datetime(transaction_history['full_date'])


In [49]:
transaction_history = transaction_history.query("description != 'Від'")

In [50]:
train_data = transaction_history.query('full_date < "2024-11-01"')
test_data = transaction_history.query('full_date >= "2024-11-01"')
 

In [51]:
transaction_history.to_csv("../datasets/transaction_history.csv")

In [52]:
transaction_history.nunique() 

Unnamed: 0      344
time            402
description      85
amount          245
full_date       150
month_number     12
week_number      39
dtype: int64

In [58]:
transaction_history.groupby('description').nunique().sort_values('amount',ascending=False).head(40)

,Unnamed: 0,time,amount,full_date,month_number,week_number
description,,,,,,
Glovo,65,65,47,55,7,22
Uklon,63,64,39,32,5,16
414950****4044,46,47,26,37,12,24
Simi,21,21,19,18,7,12
1minute - експрес маркет,11,11,9,10,4,10
McDonald’s,9,9,9,9,4,7
SIMI #4133,8,8,7,8,5,7
З доларової картки,7,7,7,7,6,6
ZERNO,7,7,6,7,4,5


### For Labeled data

In [31]:
target_variable = "description"
report = sv.compare([train_data, "Train"], [test_data, "Test"], target_feat=target_variable)
report_path = 'Comparative_EDA_Report.html'
report.show_html(filepath=report_path, open_browser=False)

                                             |          | [  0%]   00:00 -> (? left)

ValueError: TARGET values can only be of NUMERICAL or BOOLEAN type for now.
CATEGORICAL type was detected; if you meant the target to be
NUMERICAL, use a FeatureConfig(force_num=...) object.